<a href="https://colab.research.google.com/github/harry9879/Tensorflow_Tutorial_for_text/blob/main/Mask/Masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

padding_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs , padding = 'post'
)
print(padding_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [ ]:
embedding = layers.Embedding(input_dim = 5000,output_dim = 16, mask_zero = True)
masked_output = embedding(padding_inputs)

print(masked_output._keras_mask)
masking_layer = layers.Masking()

unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padding_inputs, axis= -1), [1,1,10]), tf.float32
)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)
tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


In [ ]:
model = keras.Sequential(
    [layers.Embedding(input_dim=5000, output_dim = 16, mask_zero = True), layers.LSTM(32)]
)

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int32')
x = layers.Embedding(input_dim=5000, output_dim = 16, mask_zero = True)(inputs)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

In [ ]:
class MyLayer(layers.Layer):
  def __init__(self, **kwargs):
    super(MyLayer, self).__init__(**kwargs)
    self.embedding = layers.Embedding(input_dim= 5000, output_dim = 16 , mask_zero= True)
    self.LSTM = layers.LSTM(32)

  def call(self, inputs):
    x = self.embedding(inputs)
    mask = self.embedding.compute_mask(inputs)
    output = self.LSTM(x, mask = mask)
    return output

layer = MyLayer()
x = np.random.random((32,18)) * 100
x = x.astype('int32')
layer(x)

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[-3.4250151e-03,  1.0624408e-02, -3.8551725e-03, ...,
        -6.9048353e-03,  1.0449134e-03,  1.6931095e-03],
       [ 9.2278398e-04, -4.6851030e-03,  2.8323641e-04, ...,
         2.1643487e-03,  1.4104871e-03,  6.0918122e-03],
       [-2.1287345e-03, -1.0787015e-03, -1.0347900e-02, ...,
        -4.8352797e-03, -1.7620426e-03, -2.7737911e-03],
       ...,
       [ 2.7558473e-03,  1.8737143e-03,  3.8235360e-03, ...,
         1.8294882e-03,  2.2192276e-03,  1.5437567e-03],
       [ 1.1458315e-03, -8.9187436e-03,  7.4250605e-03, ...,
        -4.5560873e-03, -5.7541928e-03,  3.1083077e-03],
       [-2.5589557e-03, -6.9083169e-04, -7.2421753e-05, ...,
         3.3397579e-03,  4.2279446e-03, -1.6200483e-03]], dtype=float32)>

In [ ]:
class TemporalSplit(keras.layers.Layer):
    """Split the input tensor into 2 tensors along the time dimension."""

    def call(self, inputs):
        # Expect the input to be 3D and mask to be 2D, split the input tensor into 2
        # subtensors along the time axis (axis 1).
        return tf.split(inputs, 2, axis=1)

    def compute_mask(self, inputs, mask=None):
        # Also split the mask into 2 if it presents.
        if mask is None:
            return None
        return tf.split(mask, 2, axis=1)


first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

tf.Tensor(
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]], shape=(3, 3), dtype=bool)
tf.Tensor(
[[False False False]
 [ True  True False]
 [ True  True  True]], shape=(3, 3), dtype=bool)


In [ ]:
class CustomEmbedding(keras.layers.Layer):
  def __init__(self, input_dim , output_dim , mask_zero = False, **kwargs):
    super(CustomEmbedding, self).__init__(**kwargs)
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.mask_zero = mask_zero

  def build(self, input_shape):
    self.embeddings = self.add_weight(
        shape = (self.input_dim , self.output_dim),
        initializer = 'random_normal',
        dtype = 'float32'
    )
  
  def call(self, inputs):
    return tf.nn.embedding_lookup(self.embeddings, inputs)
  
  def compute_mask(self, inputs, mask = None):
    if not self.mask_zero:
        return None
    return tf.not_equal(inputs, 0)

layer = CustomEmbedding(10,32, mask_zero = True)
x = np.random.random((3,10)) * 9 
x = x.astype('int32')

y = layer(x)
mask = layer.compute_mask(x)

print(mask)

tf.Tensor(
[[ True  True  True  True  True  True  True False  True  True]
 [False  True  True  True  True  True  True False  True  True]
 [ True  True  True  True  True  True  True  True False  True]], shape=(3, 10), dtype=bool)


In [ ]:
class MyActivation(keras.layers.Layer):
  def __init__(self, **kwargs):
    super(MyActivation, self).__init__(**kwargs)
    self.supports_masking = True

  def call(self, inputs):
    return tf.nn.relu(inputs)

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int32')
x = layers.Embedding(input_dim = 5000, output_dim = 16, mask_zero = True)(inputs)
x = MyActivation()(x)
print('input', inputs)
print("Mask found:", x._keras_mask)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

input KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.int32, name='input_2'), name='input_2', description="created by layer 'input_2'")
Mask found: KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.bool, name=None), name='Placeholder_1:0')


In [ ]:
class TemporalSoftmax(keras.layers.Layer):
  def call(self, inputs , mask = None):
    broadcast_float_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
    input_exp = tf.exp(inputs) * broadcast_float_mask
    inputs_sum = tf.reduce_sum(
        input_exp * broadcast_float_mask , axis = -1 , keepdims = True
    )
    return input_exp / inputs_sum

inputs = keras.Input(shape = (None,), dtype = 'int32')
x = layers.Embedding(input_dim = 10 , output_dim = 32, mask_zero= True)(inputs)
x = layers.Dense(1)(x)
outputs = TemporalSoftmax()(x)

model = keras.Model(inputs, outputs)
y = model(np.random.randint(0 , 10 , size = (32, 100)), np.random.random((32,100,1)))